In [22]:
import pandas as pd
import numpy as np
import pickle
from sklearn import ensemble
from sklearn import model_selection,tree,linear_model
from sklearn.feature_selection import SelectFromModel
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

In [23]:
hrm=pd.read_csv("data/hrm.csv")
X=hrm.drop(["Attrition","Gender","PerformanceRating","PerformanceRating"],axis=1).values
y=hrm['PerformanceRating'].values

In [24]:
feature_selector=ensemble.ExtraTreesClassifier().fit(X,y)
model=SelectFromModel(feature_selector,prefit=True)
X_new=model.transform(X)
feature_count=X_new.shape[1]

X_train,X_test,y_train,y_test=model_selection.train_test_split(X_new,y,test_size=0.40)



In [25]:
model_results ={}

model_params={
    GaussianNB():{},
    
    tree.DecisionTreeClassifier():{
        'max_depth':list(range(1,20)),
        'max_features':list(range(1,feature_count+1))
    },
    MLPClassifier():{
        'activation':['tanh','relu','logistic'],
        'solver':['sgd','adam'],
        'alpha':[1e-4,1e-3,1e-2],
        'max_iter':[10,15,20]
    },
    ensemble.RandomForestClassifier():{
        'n_estimators':[10,100,1000],
        'max_depth':list(range(1,20)),
        'max_features':list(range(1,feature_count+1))
    }
}

In [26]:
from sklearn.grid_search import GridSearchCV

for model,params in model_params.iteritems():
    grid=GridSearchCV(estimator=model,param_grid=params,n_jobs=4)
    grid.fit(X_train,y_train)
    
    test_score=grid.best_estimator_.score(X_test,y_test)
    model_results[grid.best_estimator_]=test_score

/home/akinyalcinkaya/anaconda2/envs/akinyalcinkaya/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/akinyalcinkaya/anaconda2/envs/akinyalcinkaya/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/akinyalcinkaya/anaconda2/envs/akinyalcinkaya/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/akinyalcinkaya/anaconda2/envs/akinyalcinkaya/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: Conv

In [27]:
best_model=max(model_results,key=model_results.get)

print best_model
print model_results[best_model]

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=1, max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
1.0
